In [2]:
from snowexsql.db import get_db, get_table_attributes
from snowexsql.data import ImageData

from sqlalchemy.sql import func
import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc

from sqlalchemy.sql import func
from snowexsql.conversions import raster_to_rasterio
from rasterio.plot import show

In [3]:
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

In [4]:
#lidar and uavsar data both in ImageData table/classes
qry = session.query(ImageData.instrument).distinct()
results = qry.all()
print(', '.join([row[0] for row in list(results)]))
session.close()

UAVSAR, L-band InSAR, lidar


In [5]:
img_db_columns = get_table_attributes(ImageData)
print(', '.join([row for row in list(img_db_columns)]))

date, date_accessed, description, doi, instrument, metadata, observers, raster, registry, site_name, time_created, time_updated, type, units


In [6]:
#just want UAVSAR and LiDAR
#joint : qry.filter(ImageData.instrument.in_(['lidar', 'UAVSAR']))
uavsar_db = qry.filter(ImageData.instrument == 'UAVSAR')
lidar_db = qry.filter(ImageData.instrument == 'lidar')

In [13]:
# GM polygon:

#snagged these from somewhere (sliderule GM notebook?), but would replace
#with bounds from uavsar rasters/elsewhere
gm_bds = [(39.34603060272382, -108.40601489205419),
          (39.32770853617356, -107.68485163209928),
          (39.32770853617356, -107.68485163209928),
          (38.788639821001155, -108.42635020791396),
          (39.34603060272382, -108.40601489205419)]

gm_poly = shapely.geometry.Polygon(gm_bds)


#converting shapefile to srid 26912 (UTM 12N) for db querying:
import pyproj
from shapely.ops import transform

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:26912')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
utm_poly = transform(project, gm_poly)

#from seth:
# wkb_element = from_shape(utm_poly, srid=26912)

#basic bbox, since not getting anything w/ complex polygon object:
#(count = 0) after query
w,s,e,n= -108.42635020791396, 38.788639821001155, -107.68485163209928, 39.34603060272382
bbox = shapely.geometry.Polygon([[w, s], [w, n], [e, n], [e, s]])
bbox = transform(project, bbox)
bbox_wkb = from_shape(bbox, srid=26912)

In [17]:
gm_images #<sqlalchemy.orm.query.Query at 0x7fdd3a9baf50>

gm_images = session.query(func.ST_AsTiff(ImageData.raster)) #count=470646
gm_images = gm_images.filter(ImageData.instrument.in_(['UAVSAR', 'lidar'])) #7046
gm_images = gm_images.filter(func.ST_Clip(ImageData.raster, bbox_wkb)) #7046

result = gm_images.limit(5)
datasets = raster_to_rasterio(session, result)
show(datasets[0], cmap='winter')
# datasets[0].close()
session.close()

result = gm_images.all()
print(result)

aso_2020_02_13_raster = raster_to_rasterio(session, result)[0].read(1, masked=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/snowexsql/conversions.py:99: SAWarning: Class WKBElement will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  for r in rasters:


PendingRollbackError: Can't reconnect until invalid transaction is rolled back. (Background on this error at: https://sqlalche.me/e/14/8s2b)

In [ ]:
# build an envelope from one dataset and then use it to filter another
# dataset to the same bounds

# **ST_Envelope** — Returns the polygon representation of the extent of the raster.

#ST_Envelope — Returns the polygon representation of the extent of the raster.
#ST_DumpAsPolygons? #https://postgis.net/docs/RT_ST_DumpAsPolygons.html

In [9]:
gm_images.count()

/tmp/ipykernel_196/2783810327.py:1: SAWarning: UserDefinedType Raster() will not produce a cache key because the ``cache_ok`` attribute is not set to True.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this type object's state is safe to use in a cache key, or False to disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  gm_images.count()


7046

In [ ]:
# initialize list of envelopes of polygons:
#list_envelopes

# for raster in gm_images:
    
    #geom = ST_Envelope(raster)
    #list_envelopes+=geom

In [ ]:
#TODOs:
    # only rasters for lidar data? no uavsar in db?
    # ST_Intersect doesn't seem to be paring down list of lidar at all
    # envelopes so that can search other data types